In [2]:
import pandas as pd
import numpy as np

In [17]:
data_path = 'F:/Documents/CRCE/Project/NIDS/dataset/Edge-IIoT/Edge-IIoTset dataset/Selected dataset for ML and DL/DNN-EdgeIIoT-dataset.csv'  # Replace with your actual path
df = pd.read_csv(data_path, low_memory=False)
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,2021 11:44:10.081753000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
1,2021 11:44:10.162218000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,4.0,MQTT,0,0.0,4.0,0.0,0.0,0.0,0,Normal
2,2021 11:44:10.162271000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
3,2021 11:44:10.162641000,192.168.0.128,192.168.0.101,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,Normal
4,2021 11:44:10.166132000,192.168.0.101,192.168.0.128,0,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,Temperature_and_Humidity,24.0,0.0,0.0,0.0,0.0,0,Normal


In [3]:
df.head(5)
print(df['Attack_type'].value_counts())

Attack_type
Normal                   1615643
DDoS_UDP                  121568
DDoS_ICMP                 116436
SQL_injection              51203
Password                   50153
Vulnerability_scanner      50110
DDoS_TCP                   50062
DDoS_HTTP                  49911
Uploading                  37634
Backdoor                   24862
Port_Scanning              22564
XSS                        15915
Ransomware                 10925
MITM                        1214
Fingerprinting              1001
Name: count, dtype: int64


In [16]:
from sklearn.utils import shuffle

drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 
         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",
         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",
         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(subset=None, keep="first", inplace=True)
df = shuffle(df)
df.isna().sum()

print(df['Attack_type'].value_counts())

Attack_type
Normal                   1363998
DDoS_UDP                  121567
DDoS_ICMP                  67939
SQL_injection              50826
DDoS_TCP                   50062
Vulnerability_scanner      50026
Password                   49933
DDoS_HTTP                  48544
Uploading                  36807
Backdoor                   24026
Port_Scanning              19977
XSS                        15066
Ransomware                  9689
Fingerprinting               853
MITM                         358
Name: count, dtype: int64


In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

encode_text_dummy(df,'http.request.method')
encode_text_dummy(df,'http.referer')
encode_text_dummy(df,"http.request.version")
encode_text_dummy(df,"dns.qry.name.len")
encode_text_dummy(df,"mqtt.conack.flags")
encode_text_dummy(df,"mqtt.protoname")
encode_text_dummy(df,"mqtt.topic")

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Step 3: Encode labels
label_encoder = {label: idx for idx, label in enumerate(df["Attack_type"].unique())}
df["Attack_type"] = df["Attack_type"].map(label_encoder)

# Step 4: Split features and target
X = df.drop(["Attack_label", "Attack_type"], axis=1).values
y = df["Attack_type"].values

# Step 5: Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 7: Reshape for CNN input: (batch_size, channels, features)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

num_class = 15

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(32, 16, kernel_size=3, padding=1)        
        self.fc1 = nn.Linear(95 * 16, 30)
        self.out_layer = nn.Linear(30, num_class)
        self.relu = nn.ReLU()
    def forward(self, x):
        # conv layer 1
        x = self.conv1(x)
        x = F.relu(x)
        # conv layer 2
        x = self.conv2(x)
        x = F.relu(x)
        # conv layer 3
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool1d(x, kernel_size=1)
        # Flatten layer 
        x = x.reshape(x.size(0), -1)
        # fc layer 1
        x = self.fc1(x)
        x = F.relu(x)
        # Output Layer 
        x = self.out_layer(x)
        output = F.log_softmax(x, dim=1)
        return output

In [10]:
import sys
from sklearn.metrics import accuracy_score, classification_report


for epoch in range(10):
    model.train()
    total_loss = 0
    num_batches = len(train_loader)

    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Overwrite the batch number on the same line
        print(f"\rEpoch {epoch+1} | Batch {batch_idx+1}/{num_batches}", end='', flush=True)
    print(f"\nTotal Loss: {total_loss:.4f}")


    # Evaluation after each epoch
    model.eval()
    all_preds = []
    all_labels = []

    acc = accuracy_score(all_labels, all_preds)
    print(f"\nTotal Loss: {total_loss:.4f} ")
    # | Validation Accuracy: {acc*100:.2f}%")
    print(classification_report(all_labels, all_preds, digits=4))


NameError: name 'model' is not defined

In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Step 1: Clean & shuffle the DataFrame
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4", "arp.dst.proto_ipv4",
                "http.file_data", "http.request.full_uri", "icmp.transmit_timestamp", "http.request.uri.query",
                "tcp.options", "tcp.payload", "tcp.srcport", "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df.dropna(axis=0, how='any', inplace=True)
df.drop_duplicates(inplace=True)
df = shuffle(df)

# Step 2: One-hot encoding
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name], prefix=name)
    df = pd.concat([df, dummies], axis=1)
    df.drop(name, axis=1, inplace=True)
    return df

for col in ["http.request.method", "http.referer", "http.request.version",
            "dns.qry.name.len", "mqtt.conack.flags", "mqtt.protoname", "mqtt.topic"]:
    df = encode_text_dummy(df, col)

# Step 3: Encode labels
label_encoder = {label: idx for idx, label in enumerate(df["Attack_type"].unique())}
df["Attack_type"] = df["Attack_type"].map(label_encoder)

# Step 4: Split features and target
X = df.drop(["Attack_label", "Attack_type"], axis=1).values
y = df["Attack_type"].values

# Step 5: Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 7: Reshape for CNN input: (batch_size, channels, features)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Step 8: Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(32, 16, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(95 * 16, 30)  # 95 is input feature count, adjust if needed
        self.out_layer = nn.Linear(30, 15)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool1d(x, kernel_size=1)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.out_layer(x), dim=1)

# Step 9: Train the CNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

import sys
from sklearn.metrics import accuracy_score, classification_report


for epoch in range(10):
    model.train()
    total_loss = 0
    num_batches = len(train_loader)

    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Overwrite the batch number on the same line
        print(f"\rEpoch {epoch+1} | Batch {batch_idx+1}/{num_batches}", end='', flush=True)
    print(f"\nTotal Loss: {total_loss:.4f}")


    # Evaluation after each epoch
    model.eval()
    all_preds = []
    all_labels = []

    # with torch.no_grad():
    #     for batch_X, batch_y in val_loader:
    #         batch_X, batch_y = batch_X.to(device), batch_y.to(device)
    #         output = model(batch_X)
    #         preds = torch.argmax(output, dim=1)
    #         all_preds.extend(preds.cpu().numpy())
    #         all_labels.extend(batch_y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"\nTotal Loss: {total_loss:.4f} ")
    # | Validation Accuracy: {acc*100:.2f}%")
    print(classification_report(all_labels, all_preds, digits=4))


Epoch 1 | Batch 23871/23871
Total Loss: 2742.7554

Total Loss: 2742.7554 


C:\Users\rosha\Documents\Jupyter\NIDS\new_venv\adi-venv\Lib\site-packages\numpy\lib\_function_base_impl.py:552: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\rosha\Documents\Jupyter\NIDS\new_venv\adi-venv\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


ValueError: max() iterable argument is empty

In [9]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming df is already preprocessed and encoded

# Separate features and labels
X = df.drop(["Attack_label", "Attack_type"], axis=1).values
y = df["Attack_label"].values  # or a different target if you're using multiclass

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # shape (batch, 1, features)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [10]:
from sklearn.metrics import accuracy_score, classification_report

for epoch in range(10):
    model.train()
    total_loss = 0
    num_batches = len(train_loader)

    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Print live batch progress like: Epoch 1 | Batch 12/87
        print(f"\rEpoch {epoch+1} | Batch {batch_idx+1}/{num_batches}", end='', flush=True)

    # Evaluation after each epoch
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            output = model(batch_X)
            preds = torch.argmax(output, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"\nTotal Loss: {total_loss:.4f} | Validation Accuracy: {acc*100:.2f}%")
    print(classification_report(all_labels, all_preds, digits=4))


Epoch 1 | Batch 23871/23871
Total Loss: 45.9101 | Validation Accuracy: 100.00%
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000    272800
           1     1.0000    1.0000    1.0000    109135

    accuracy                         1.0000    381935
   macro avg     1.0000    1.0000    1.0000    381935
weighted avg     1.0000    1.0000    1.0000    381935

Epoch 2 | Batch 2813/23871

KeyboardInterrupt: 